In [ ]:
# ==============================================
# Testing_validation.ipynb (оновлена версія з GT)
# ==============================================

import cv2
import numpy as np
import tensorflow as tf
from ultralytics import YOLO
import matplotlib.pyplot as plt
import os
import glob

# ----------------------------------------------
# TODO: вкажіть ваші шляхи
# ----------------------------------------------
YOLO_MODEL_PATH = "./runs/detect/train9/weights/best.pt"
CLASSIFIER_MODEL_PATH = "./models/digit_classifier_best.keras"
TEST_IMAGES_DIR = "./learning/data_number/valid/images"
TEST_LABELS_DIR = "./learning/data_number/valid/labels"
# ----------------------------------------------

# Завантаження моделей
yolo_model = YOLO(YOLO_MODEL_PATH)
classifier = tf.keras.models.load_model(CLASSIFIER_MODEL_PATH)

# ----------------------------------------------
# Функція препроцесингу цифри (padding + grayscale)
# ----------------------------------------------
def preprocess_digit_for_classifier(img, target_size=(64, 64)):
    if len(img.shape) == 3:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img = cv2.GaussianBlur(img, (3,3), 0)
    img = cv2.equalizeHist(img)

    # опціонально, якщо фон темний:
    # _, img = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    h, w = img.shape[:2]
    if h == 0 or w == 0:
        return np.zeros((*target_size, 1), dtype=np.float32)

    scale = min(target_size[0] / h, target_size[1] / w)
    new_w, new_h = max(1, int(w * scale)), max(1, int(h * scale))
    resized = cv2.resize(img, (new_w, new_h), interpolation=cv2.INTER_AREA)

    dw = target_size[1] - new_w
    dh = target_size[0] - new_h
    top, bottom = dh // 2, dh - dh // 2
    left, right = dw // 2, dw - dw // 2

    padded = cv2.copyMakeBorder(resized, top, bottom, left, right, cv2.BORDER_CONSTANT, value=0)
    padded = padded.astype('float32') / 255.0  # якщо немає Rescaling
    padded = np.expand_dims(padded, axis=-1)
    return padded


# ----------------------------------------------
# Функція для класифікації одного кропа
# ----------------------------------------------
def classify_digit(crop):
    processed = preprocess_digit_for_classifier(crop)
    pred = classifier.predict(np.expand_dims(processed, axis=0), verbose=0)
    return np.argmax(pred)

# ----------------------------------------------
# Читання ground truth з YOLO label файлу
# ----------------------------------------------
def load_ground_truth_from_txt(label_path):
    digits = []
    if not os.path.exists(label_path):
        return digits
    with open(label_path, "r") as f:
        for line in f:
            parts = line.strip().split()
            if len(parts) >= 5:
                cls_id = int(float(parts[0]))
                x_center = float(parts[1])
                digits.append((x_center, cls_id))
    # сортуємо за координатою X (YOLO дає нормалізовані)
    digits.sort(key=lambda x: x[0])
    gt_digits = [str(int(d[1])) for d in digits]
    return gt_digits

# ----------------------------------------------
# Обробка одного зображення
# ----------------------------------------------
def process_image(image_path, visualize=True):
    image = cv2.imread(image_path)
    if image is None:
        print(f"❌ Не вдалося завантажити: {image_path}")
        return None, None

    # YOLO детекція
    results = yolo_model.predict(image, verbose=False)
    boxes = results[0].boxes.xyxy.cpu().numpy() if len(results) > 0 else []

    if len(boxes) == 0:
        print(f"⚠️ Нічого не знайдено на {os.path.basename(image_path)}")
        return [], image

    boxes = sorted(boxes, key=lambda x: x[0])  # сортування за X

    predicted_digits = []

    for (x1, y1, x2, y2) in boxes:
        x1, y1, x2, y2 = map(int, [x1, y1, x2, y2])
        if (x2 - x1) < 5 or (y2 - y1) < 5:
            continue
        crop = image[y1:y2, x1:x2]
        if crop.size == 0:
            continue
        digit = classify_digit(crop)
        predicted_digits.append(str(digit))

        if visualize:
            cv2.rectangle(image, (x1, y1), (x2, y2), (0,255,0), 2)
            cv2.putText(image, str(digit), (x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,255,0), 2)

    if visualize:
        plt.figure(figsize=(8, 6))
        plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        plt.title(f"Predicted: {''.join(predicted_digits)}")
        plt.axis('off')
        plt.show()

    return predicted_digits, image

# ----------------------------------------------
# Оцінка на тестових зображеннях
# ----------------------------------------------
correct = 0
total = 0

for image_path in sorted(glob.glob(os.path.join(TEST_IMAGES_DIR, "*.*"))):
    if not image_path.lower().endswith((".png", ".jpg", ".jpeg")):
        continue

    file_id = os.path.splitext(os.path.basename(image_path))[0]
    label_path = os.path.join(TEST_LABELS_DIR, file_id + ".txt")

    gt_digits = load_ground_truth_from_txt(label_path)
    predicted_digits, _ = process_image(image_path, visualize=False)

    if not gt_digits:
        print(f"⚠️ Немає label для {file_id}")
        continue

    # Порівняння
    gt_str = "".join(gt_digits)
    pred_str = "".join(predicted_digits)
    match = (gt_str == pred_str)
    if match:
        correct += 1
    total += 1
    if total == 100:
        break
    print(f"{file_id} | GT={gt_str} | Pred={pred_str} | {'✅' if match else '❌'}")

# ----------------------------------------------
# Фінальна точність
# ----------------------------------------------
if total > 0:
    acc = correct / total * 100
    print(f"\n✅ Accuracy: {acc:.2f}% ({correct}/{total})")
else:
    print("⚠️ Не знайдено тестових зображень.")


img_00000 | GT=54 | Pred=54 | ✅
img_00001 | GT=4052 | Pred=40552 | ❌
img_00002 | GT=749 | Pred=748 | ❌
img_00003 | GT=4135 | Pred=4888 | ❌
img_00004 | GT=5 | Pred=5 | ✅
img_00005 | GT=94 | Pred=94 | ✅
img_00006 | GT=4111 | Pred=41111 | ❌
img_00007 | GT=5348 | Pred=5348 | ✅
img_00008 | GT=8416 | Pred=347 | ❌
img_00009 | GT=44 | Pred=44 | ✅
img_00010 | GT=1749 | Pred=1749 | ✅
img_00011 | GT=881 | Pred=887 | ❌
img_00012 | GT=6481 | Pred=648 | ❌
img_00013 | GT=59 | Pred=59 | ✅
img_00014 | GT=48 | Pred=48 | ✅
img_00015 | GT=80293 | Pred=30293 | ❌
img_00016 | GT=10804 | Pred=1080 | ❌
img_00017 | GT=4 | Pred=4 | ✅
img_00018 | GT=85511 | Pred=855111 | ❌
img_00019 | GT=1785 | Pred=1785 | ✅
img_00020 | GT=45064 | Pred=45064 | ✅
img_00021 | GT=38 | Pred=38 | ✅
img_00022 | GT=0950 | Pred=0950 | ✅
img_00023 | GT=24 | Pred=24 | ✅
img_00024 | GT=74686 | Pred=74686 | ✅
img_00025 | GT=2 | Pred=2 | ✅
img_00026 | GT=349 | Pred=848 | ❌
img_00027 | GT=83 | Pred=833 | ❌
img_00028 | GT=2 | Pred=8 | ❌
img_000